In [37]:
%pip install pymysql

In [38]:
import pymysql

In [39]:
def read_config(p:str) -> dict:
    """
    p: config file 경로
    """ 
    with open(p,'r') as f:
        lines = f.readlines()
    config_dict={}

    for l in lines:
        idx = l.index('=')
        k = l[:idx]
        v = l[idx+1:]
        config_dict[k] = v.rstrip()


    return config_dict

In [42]:
db_config = read_config('./test.config')

In [45]:
# config 파일 관리하는 코드
with open('./DB_config','r') as f:
    config = f.readlines()

In [ ]:
config_dict = {} 

for l in config:
    # k, v = l.split('=')   밑에 3개랑 같은 의미인데 혹시 구분이 ex. host=jksdfk@dnf=dkfj 이런 식이면 앞에 = 로 구분하고 싶은데 = 가 두개라서 3 개로 구분되어서 안 되는 경우 때문에 밑에 처럼 해서 그런 경우 안 나오게 함
    idx = l.index('=')
    k = l[:idx]
    v = l[idx+1:]
    config_dict[k] = v.rstrip()

config_dict

In [47]:
try:
    # conn = pymysql.connect(
    #     host="localhost", 
    #     # port=3306, 생략가능
    #     user="내 유저네임 작성하면 됨",
    #     password="유저네임에 대한 비밀번호 작성하면 됨",
    #     database="study",
    #     charset="utf8mb4"
    # )
    conn = pymysql.connect(**config_dict)
    # conn = pymysql.connect(**db_config)
    # conn = pymysql.connect(host='localhost', user='JUNHWA')

    print("연결 성공")
except Exception as e:
    print("연결 실패", e)

연결 성공


In [48]:
conn

### INSERT

In [15]:
with conn.cursor() as cursor:
    sql = 'INSERT INTO `Students`(`Name`, `Age`, `Address`) \
        VALUES ("정준화", "27", "서울")' 
    cursor.execute(sql)

conn.commit() 

In [16]:
with conn.cursor() as cursor:
    sql = 'INSERT INTO `Students`(`Name`, `Age`, `Address`) \
        VALUES (%s, %s, %s)' 
    cursor.execute(sql, ("최용훈", "30", "서울"))

conn.commit() 

In [17]:
with conn.cursor() as cursor:
    sql = 'INSERT INTO `Students`(`Name`) \
        VALUES (%s)' 
    cursor.execute(sql, ("정우경", ))    # 이렇게 뒤에 , 붙여서 튜플로 만들어주기.

conn.commit() 

In [18]:
type(("정우경"))

str

In [19]:
type(("정우경", ))

tuple

### INSERT MANY

In [20]:
with conn.cursor() as cursor:
    sql = 'INSERT INTO `Students`(`Name`) \
        VALUES (%s)' 
    cursor.executemany(sql, ("정우경", ))

conn.commit() 

In [21]:
with conn.cursor() as cursor:
    sql = 'INSERT INTO `Students`(`Name`) \
        VALUES (%s)' 
    cursor.executemany(sql, [
        ("서지은", ),   # data1
         ("정찬영", )   # data2
    ])

conn.commit() 

### SELECT

In [22]:
with conn.cursor() as cur: 
    sql = 'SELECT * FROM Students'
    cur.execute(sql) 
    data = cur.fetchone() 

data

(1, '홍길동', 24, '인천')

In [23]:
with conn.cursor() as cur: 
    sql = 'SELECT * FROM Students'
    cur.execute(sql) 
    data = cur.fetchmany(size=3) 

data

((1, '홍길동', 24, '인천'), (2, '이연결', 60, '서울'), (3, '이몽룡', 30, '대전'))

In [24]:
with conn.cursor() as cur: 
    sql = 'SELECT * FROM Students'
    cur.execute(sql) 
    data = cur.fetchall() 

data

((1, '홍길동', 24, '인천'),
 (2, '이연결', 60, '서울'),
 (3, '이몽룡', 30, '대전'),
 (4, '성춘향', 27, '경기'),
 (6, '정준화', 27, '서울'),
 (7, '최용훈', 30, '서울'),
 (9, '정우경', None, None),
 (10, '서지은', 26, None),
 (11, '정찬영', None, None),
 (12, '정준화', 27, '서울'),
 (13, '최용훈', 30, '서울'),
 (14, '정우경', None, None),
 (15, '정우경', None, None),
 (16, '서지은', None, None),
 (17, '정찬영', None, None))

In [25]:
with conn.cursor() as cur: 
    sql = 'SELECT * FROM Students LIMIT 3 OFFSET 0'
    cur.execute(sql) 
    data = cur.fetchall() 

data

((1, '홍길동', 24, '인천'), (2, '이연결', 60, '서울'), (3, '이몽룡', 30, '대전'))

In [26]:
with conn.cursor() as cur: 
    sql = 'SELECT * FROM Students LIMIT %s OFFSET %s'
    cur.execute(sql, (5,0)) 
    data = cur.fetchall() 

data

((1, '홍길동', 24, '인천'),
 (2, '이연결', 60, '서울'),
 (3, '이몽룡', 30, '대전'),
 (4, '성춘향', 27, '경기'),
 (6, '정준화', 27, '서울'))

In [27]:
page_size = 5
page = 1

limit = page_size 
offset = page_size * (page-1)

with conn.cursor() as cur: 
    sql = 'SELECT * FROM Students LIMIT %s OFFSET %s'
    cur.execute(sql, (limit, offset)) 
    data = cur.fetchall() 

data

((1, '홍길동', 24, '인천'),
 (2, '이연결', 60, '서울'),
 (3, '이몽룡', 30, '대전'),
 (4, '성춘향', 27, '경기'),
 (6, '정준화', 27, '서울'))

In [28]:
def pagination(page, page_size=5):
    limit = page_size 
    offset = page_size * (page-1)

    with conn.cursor() as cur: 
        sql = 'SELECT * FROM Students LIMIT %s OFFSET %s'
        cur.execute(sql, (limit, offset)) 
        data = cur.fetchall() 

    return data

page_one = pagination(2)
page_one 

((7, '최용훈', 30, '서울'),
 (9, '정우경', None, None),
 (10, '서지은', 26, None),
 (11, '정찬영', None, None),
 (12, '정준화', 27, '서울'))

### UPDATE

In [29]:
with conn.cursor() as cur:
    sql = 'UPDATE `Students` SET Age=%s WHERE StudentID=%s' 
    cur.execute(sql, (26, 10))  # 순서 중요!
    
conn.commit()

In [30]:
pagination(2)

((7, '최용훈', 30, '서울'),
 (9, '정우경', None, None),
 (10, '서지은', 26, None),
 (11, '정찬영', None, None),
 (12, '정준화', 27, '서울'))

In [31]:
with conn.cursor() as cur:
    sql = 'UPDATE `Students` SET Age=%s WHERE StudentID=%s' 
    cur.executemany(sql, [
        (24, 1),
        (30, 3)
    ])  # 순서 중요!
    
conn.commit()

In [32]:
pagination(1)

((1, '홍길동', 24, '인천'),
 (2, '이연결', 60, '서울'),
 (3, '이몽룡', 30, '대전'),
 (4, '성춘향', 27, '경기'),
 (6, '정준화', 27, '서울'))

### DELETE

In [33]:
with conn.cursor() as cur: 
    sql = 'DELETE FROM `Students` WHERE StudentID=%s'
    cur.execute(sql, (5, ))

conn.commit()

In [34]:
pagination(1)

((1, '홍길동', 24, '인천'),
 (2, '이연결', 60, '서울'),
 (3, '이몽룡', 30, '대전'),
 (4, '성춘향', 27, '경기'),
 (6, '정준화', 27, '서울'))

In [35]:
# 외래키 제약조건 해제
# SET FOREIGN_KEY_CHECKS = 0;

with conn.cursor() as cur:
    sql = 'SET FOREIGN_KEY_CHECKS = 0'
    cur.execute(sql)

conn.commit()

In [36]:
with conn.cursor() as cur: 
    sql = 'DELETE FROM `Students` WHERE StudentID IN (%s, %s, %s)'
    cur.execute(sql, (2, 3, 4))

conn.commit()



In [37]:
pagination(1)

((1, '홍길동', 24, '인천'),
 (6, '정준화', 27, '서울'),
 (7, '최용훈', 30, '서울'),
 (9, '정우경', None, None),
 (10, '서지은', 26, None))

In [38]:
','.join(['%s']*3)

'%s,%s,%s'

In [40]:
def delete(target_ids):
    with conn.cursor() as cur: 
        sql = 'DELETE FROM `Students` WHERE StudentID IN (' +\
             ','.join(['%s'] * len(target_ids)) + ')'
        cur.execute(sql, target_ids)

    conn.commit()
    return

delete((2, 5, ))

In [41]:
delete((7, ))  # 한 명 지울 때 7만 적어주면 len(7) 이 오류 반환 

In [42]:
pagination(1)

((1, '홍길동', 24, '인천'),
 (6, '정준화', 27, '서울'),
 (9, '정우경', None, None),
 (10, '서지은', 26, None),
 (11, '정찬영', None, None))

In [50]:
'        \n       abcdc     \n    '.strip()    # 앞뒤로 공백을 지워주는 메서드

'abcdc'

In [51]:
'        \n       abcdc   bb  \n    '.strip()    

'abcdc   bb'

In [21]:
f = open('data.csv', 'w', encoding='utf8')

In [22]:
f

<_io.TextIOWrapper name='data.csv' mode='w' encoding='utf8'>

In [23]:
f.write("오늘의날씨")

5

In [24]:
f.close()

In [27]:
with open('data.csv', 'r', encoding="utf8") as f:
    txt = f.read()

In [28]:
print(txt)

오늘의날씨


## 실습(PyMySQL)

- db에 student 테이블을 생성해주세요.
- id (빈 값 허용 안함, 자동 증가)
- name (최대 16글자)
- email (최대 32글자)
- phone (최대 16글자)
- major (최대 16글자)

- 내 풀이, cmd 프롬포트에서 함.

CREATE TABLE student(
    id int NOT NULL AUTO_INCREMENT,
    name VARCHAR(16),
    email VARCHAR(32),
    phone VARCHAR(16),
    major VARCHAR(16),
    PRIMARY KEY(id)
);


In [ ]:
# jupyter 에서 하는 방법 

# - db에 student 테이블을 생성해주세요.
# - id (빈 값 허용 안함, 자동 증가)
# - name (최대 16글자)
# - email (최대 32글자)
# - phone (최대 16글자)
# - major (최대 16글자)

with conn.cursor() as cur:
    sql = """CREATE TABLE student(
        id int NOT NULL AUTO_INCREMENT,
        name VARCHAR(16),
        email VARCHAR(32),
        phone VARCHAR(16),
        major VARCHAR(16),
        PRIMARY KEY(id)
    )"""
    cur.execute(sql, )

conn.commit()

In [1]:
columns = ['name', 'email', 'phone', 'major']
','.join(columns)

'name,email,phone,major'

In [57]:
# - 수강생 중 한 분의 데이터를 DB에 입력해주세요.
# - 내 풀이

with conn.cursor() as cursor:
    sql = 'INSERT INTO `student`(`name`, `email`, `phone`, `major`) \
        VALUES ("박지영", "dkjfna@naver.com", "010-1111-1111", "글경")'
    cursor.execute(sql)
    
conn.commit()

In [ ]:
# - 수강생 중 한 분의 데이터를 DB에 입력해주세요.
# 다른 풀이

table_name = "student"
columns = ['name', 'email', 'phone', 'major']
','.join(columns)

values = ("최재진", "jason@fins.ai", "01099282799", "산업공학")

with conn.cursor() as cur:
    ','.join(['%s'] * len(values))
    sql = 'INSERT INTO `{}`({}) VALUES ({})'.format(table_name, 
                                                    ','.join(columns), 
                                                    ','.join(['%s'] * len(values)))

    cur.execute(sql, values)
conn.commit()

In [ ]:
# - 수강생 중 한 분의 데이터를 DB에 입력해주세요.
# 함수버전

def insert(table_name:str, columns:list, values:tuple):
    """
    table_name에 columns항목들에 values를 입력해줍니다.
    """
    try:
        with conn.cursor() as cur:
            ','.join(['%s'] * len(values))
            sql = 'INSERT INTO `{}`({}) VALUES ({})'.format(table_name, 
                                                            ','.join(columns), 
                                                            ','.join(['%s'] * len(values)))

            cur.execute(sql, values)
        conn.commit()
        return True
    except Exception as e:
        print(e)
        return False

In [ ]:
table_name = "student"
columns = ['name', 'email', 'phone', 'major']
values = ("최재진", "jason@fins.ai", "01099282799", "산업공학")

insert(table_name, columns, values)

In [58]:
# - 수강생 중 네 분의 데이터를 DB에 동시에 추가해주세요.
# 내 풀이

with conn.cursor() as cursor:
    sql = 'INSERT INTO `student`(`name`, `email`, `phone`, `major`) \
        VALUES (%s, %s, %s, %s)'
    cursor.executemany(sql, [
        ("김찬식", "kljw@gmail.com", "010-2222-2222", "전기"),
        ("박병일", "bydoj@gmail.com", "010-3333-3333", "산업경영"),
        ("이재혁", "leejh@naver.com", "010-5555-5555", "신소재공학"),
        ("남현호", "namght@gmail.com", "010-6666-6666", "경영")
    ])

conn.commit()


In [ ]:
# - 수강생 중 네 분의 데이터를 DB에 동시에 추가해주세요.

table_name = "student"
columns = ['name', 'email', 'phone', 'major']

# values = ("최재진", "jason@fins.ai", "01099282799", "산업공학")
values = [
    ("김도훈", "jason@fins.ai", "01011112222", "산업공학"),
    ("오미정", "jason@fins.ai", "01033334444", "산업공학"),
    ("김형석", "jason@fins.ai", "01055556666", "산업공학"),
    ("차민경", "jason@fins.ai", "01077778888", "산업공학")
]

table_name, ','.join(columns), ','.join(['%s'] * len(values[0]))

with conn.cursor() as cur:
    sql = 'INSERT INTO {}({}) VALUES ({})'.format(table_name, ','.join(columns), ','.join(['%s'] * len(values[0])))
    cur.executemany(sql, values)

conn.commit()

In [ ]:
# - 수강생 중 네 분의 데이터를 DB에 동시에 추가해주세요.
# 함수버전

def insert_many(table_name:str, columns: list, values:list):
    with conn.cursor() as cur:
        sql = 'INSERT INTO {}({}) VALUES ({})'.format(table_name, ','.join(columns), ','.join(['%s'] * len(values[0])))
        cur.executemany(sql, values)

    conn.commit()

In [59]:
# - 입력한 전체 데이터를 확인해주세요.
# 내 풀이

with conn.cursor() as cur:
    sql = 'SELECT * FROM student'
    cur.execute(sql)
    data = cur.fetchall()

data

((1, '박지영', 'dkjfna@naver.com', '010-1111-1111', '글경'),
 (2, '김찬식', 'kljw@gmail.com', '010-2222-2222', '전기'),
 (3, '박병일', 'bydoj@gmail.com', '010-3333-3333', '산업경영'),
 (4, '이재혁', 'leejh@naver.com', '010-5555-5555', '신소재공학'),
 (5, '남현호', 'namght@gmail.com', '010-6666-6666', '경영'))

In [ ]:
# - 입력한 전체 데이터를 확인해주세요.

columns = ["id", 'name']
columns = ['*']
table_name = "student"

with conn.cursor() as cur:
    sql = "SELECT {} FROM {}".format(','.join(columns), table_name)
    cur.execute(sql)
    data = cur.fetchall()

In [ ]:
data

In [60]:
# - 한 수강생분의 이메일이 잘못 입력되었다고 가정하고, 이메일을 수정, DB에 반영해주세요.
# 내 풀이

with conn.cursor() as cur:
    sql = 'UPDATE `student` SET email=%s WHERE id=%s'
    cur.execute(sql, ("byungil@naver.com", 3))

conn.commit()


In [ ]:
# - 한 수강생분의 이메일이 잘못 입력되었다고 가정하고, 이메일을 수정, DB에 반영해주세요.

table_name = 'student'
set_value = "hyungsuk0815@gmail.com"
target_id = 5

with conn.cursor() as cur:
    sql = 'UPDATE {} SET email=%s WHERE id={}'.format(
        table_name, target_id
    )
    cur.execute(sql, (set_value, ))
conn.commit()

In [61]:
# - 수강생 한 분이 취업하셨습니다. DB에서 삭제 해주세요.

with conn.cursor() as cur:
    sql = 'DELETE FROM `student` WHERE id=%s'
    cur.execute(sql, (4, ))

conn.commit()

## 실습 - 클래스화

In [65]:
# 기본 틀

class MariaDB:
    """
    MariaDB를 관리하는 클래스입니다.
    """ 

    def __init__(self, host):
        self.host = host

        "다른 함수에서 사용하기 위해 self에 담아준다."
        self.conn = pymysql.connect(host=host, user="")
        return 
    
    def insert(self):
        
        with self.conn.cursor() as cur:
            ...
        return
    
    def insert_many(self):
        with self.conn.cursor() as cur:
            ...
        return 
    
    def select(self):
        return
    
    def update(self):
        return 

In [64]:
# 풀이

class MariaDB:
    """
    MariaDB를 관리하는 클래스 입니다.
    """

    def __init__(self, host, user, password, database, charset, port=3306):
        self.host = host
        self.port = int(port)

        "다른함수에서 사용하기위해 self에 담아준다."
        self.conn = pymysql.connect(
                                host=host, 
                                user=user, 
                                password=password, 
                                database=database,
                                charset=charset, 
                                port=port)
        return 
    
    def insert(self, table_name:str, columns:list, values:tuple):
        with self.conn.cursor() as cur:
            sql = 'INSERT INTO `{}`({}) VALUES ({})'.format(table_name, 
                                                            ','.join(columns), 
                                                            ','.join(['%s'] * len(values)))

            cur.execute(sql, values)
        self.conn.commit()
        return True
    
    def insert_many(self, table_name:str, columns: list, values:list):
        with self.conn.cursor() as cur:
            sql = 'INSERT INTO {}({}) VALUES ({})'.format(table_name, ','.join(columns), ','.join(['%s'] * len(values[0])))
            cur.executemany(sql, values)

        self.conn.commit()
        return
    
    def select(self, table_name, columns, page=None, page_size=10): # +pagination 기능 추가
        if page:
            limit = page_size
            offset = page_size * (page-1)
            with self.conn.cursor() as cur:
                sql = 'SELECT {} FROM {} LIMIT %s OFFSET %s'.format(','.join(columns), table_name)
                cur.execute(sql, (limit, offset))
                data = cur.fetchall()
        else:
            with self.conn.cursor() as cur:
                sql = "SELECT {} FROM {}".format(','.join(columns), table_name)
                cur.execute(sql)
                data = cur.fetchall()

        return data
    
    def update(self, table_name, set_column, set_value, target_id):
        with self.conn.cursor() as cur:
            sql = 'UPDATE {} SET {}=%s WHERE id={}'.format(
                table_name, set_column, target_id
            )
            cur.execute(sql, (set_value, ))
        self.conn.commit()
        return

In [66]:
# 연결 테스트
db = MariaDB(**db_config) # test
db1 = MariaDB(**db_config) # production

In [67]:
db.conn

In [68]:
db1.conn

In [69]:
# insert 테스트
db.insert()

TypeError: insert() missing 3 required positional arguments: 'table_name', 'columns', and 'values'

In [70]:
# insert_many 테스트
db.insert_many()

TypeError: insert_many() missing 3 required positional arguments: 'table_name', 'columns', and 'values'

In [71]:
# select 테스트
db.select()

TypeError: select() missing 2 required positional arguments: 'table_name' and 'columns'

In [ ]:
# update 테스트
db.update()

In [ ]:
db.select()